In [1]:
import site

site.addsitedir("..")

from pathlib import Path

import pandas as pd
import torch

from src.ai.rollout import *
from src.ai.train import *

In [2]:
%load_ext autoreload
%aimport src.ai.train
%autoreload 2

In [3]:
outdir = Path("/Users/davidzheng/projects/crew-ai/outdirs/0323/run_19")
# outdir = Path("/Users/davidzheng/projects/crew-ai/outdirs/0323/run_7")
set_aux_info_hist_only(True)

In [4]:
settings_dict = torch.load(outdir / "settings.pth", weights_only=False)
settings = settings_dict["settings"]
hp = settings_dict["hp"]
models = get_models(hp, settings)
policy_model = models["policy"].eval()
value_model = models["value"].eval()

In [5]:
checkpoint = torch.load(outdir / "checkpoint.pth", weights_only=False)
policy_model.load_state_dict(checkpoint["policy_model"])
value_model.load_state_dict(checkpoint["value_model"])
td = checkpoint["td"]
td_train = td[: hp.num_train_rollouts_per_round]
td_val = td[hp.num_train_rollouts_per_round :]

In [6]:
aux_info_spec = get_aux_info_spec(settings)

In [7]:
td_train["values"], td_train["aux_info_preds"] = value_model(td_train["inps"])

In [56]:
td_small = TensorDict(
    {k: v for k, v in td_train.items() if k in ["aux_infos", "aux_info_preds"]}
)
td_small.auto_batch_size_()
td_small = td_small[:, -1:]

In [57]:
df = aux_info_spec.copy()
df["loss"] = pd.Series(
    compute_aux_info_loss(td_small, aux_info_spec, separate=True)
).values
df["uw_loss"] = df["loss"] / df["weight"]
df["max_loss"] = np.where(
    df["num_cat"].notna(),
    np.log(df["num_cat"]),
    td_small["aux_infos"].var(dim=(0, 1))[: len(df)],
)
df["r2"] = 1 - df["uw_loss"] / df["max_loss"]
df.set_index("name", inplace=True)
df

,num_cat,weight,loss,uw_loss,max_loss,r2
name,,,,,,
hist_tricks_p0,NaN,5.0,0.001017,0.000203,0.045436,0.995525
hist_tricks_p1,NaN,5.0,0.000698,0.000140,0.033863,0.995876
hist_tricks_p2,NaN,5.0,0.000907,0.000181,0.034666,0.994765
cards_left_s0,NaN,1.0,0.000451,0.000451,0.010878,0.958526
cards_left_s1,NaN,1.0,0.000607,0.000607,0.010566,0.942580
cards_left_s2,NaN,1.0,0.000341,0.000341,0.010764,0.968303
cards_left_s3,NaN,1.0,0.000524,0.000524,0.010492,0.950068
cards_left_s4,NaN,1.0,0.001221,0.001221,0.027484,0.955563
max_rank_left_s0,NaN,1.0,0.001914,0.001914,0.086992,0.978004


## Examples


In [23]:
y = td_small["aux_infos"][..., 0]
pred = td_small["aux_info_preds"][..., 0]

bad_idx = ((y - pred) ** 2).sum(dim=1).argmax().item()
print(bad_idx)
y = y[bad_idx]
pred = pred[bad_idx]

3990


In [24]:
y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [25]:
pred

tensor([-0.0099, -0.0032, -0.0036,  0.0004, -0.0073, -0.0077, -0.0004, -0.0112,
        -0.0031, -0.0114, -0.0095,  0.0011,  0.0004, -0.0140, -0.0069, -0.0075,
        -0.0127, -0.0155, -0.0045, -0.0080, -0.0019, -0.0067, -0.0109, -0.0114,
         0.0052,  0.0071,  0.0087,  0.0068,  0.0096,  0.0195,  0.0359,  0.0421,
         0.0409,  0.0498,  0.0542,  0.0730], grad_fn=<SelectBackward0>)

In [81]:
print(pred.reshape(-1, 3))
print(y.reshape(-1, 3))
print(td_train["inps"]["private"]["player_idx"][bad_idx].reshape((-1, 3)))

tensor([[-0.0017,  0.0030, -0.0003],
        [ 0.0079, -0.0062, -0.0059],
        [ 0.0218, -0.0044, -0.0023],
        [ 0.0010, -0.0044,  0.0115],
        [ 0.0229, -0.0041, -0.0037],
        [ 0.0037, -0.0008,  0.0106],
        [ 0.0199, -0.0032, -0.0027],
        [ 0.0080,  0.0044,  0.0158],
        [ 0.0287,  0.0087,  0.0144],
        [ 0.0202,  0.0236,  0.0371],
        [ 0.0455,  0.0403,  0.0504],
        [ 0.0466,  0.0571,  0.0790]], grad_fn=<ReshapeAliasBackward0>)
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[0, 1, 2],
        [0, 1, 2],
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0],
        [1, 2, 0]], dtype=torch.int8)
